<a href="https://colab.research.google.com/github/vin-thomas/transcript/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!sudo apt-get update && sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,494 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,027 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,265 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,292 kB]
Hit:13 https://ppa.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pdftotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pdftotext: filename=pdftotext-2.2.2-cp310-cp310-linux_x86_64.whl size=61850 sha256=afd1038e902b009aaa312f2906ba1139bc050576f70e4edbbea8574095a33a27
  Stored in directory: /root/.cache/pip/wheels/c8/68/c6/82a8c2046e1a8f9051017319dc007f43ef3a6edc53149d1f4e
Successfully built pdftotext


In [5]:
import re
import os

In [68]:
# @title Using pdftotext
import pdftotext
def text_extraction(path):
  with open(path, "rb") as f:
      pdf = pdftotext.PDF(f)
  text = []
  # All pages
  for txt in pdf:
    text.append(txt)
  return text

In [69]:
import os
texts = []
fname = []
def find_pdfs(folder_path):
    pdf_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# Provide the path of the folder you want to search for PDF files
folder_path = '/content/drive/MyDrive/fin'

pdf_paths = find_pdfs(folder_path)
for pdf_path in pdf_paths:
  fn = re.search(r'/content/(?P<extracted_part>.*?)\.pdf$', pdf_path)
  if fn:
    fname.append(fn.group('extracted_part'))
  texts.append(text_extraction(pdf_path))
  # print(pdf_path)


In [8]:
import glob

directory = '/content/drive/MyDrive/fin'
pattern = directory + '/*.pdf'

texts = []
fname = []
file_list = glob.glob(pattern)

for file in file_list:
  fn = re.search(r'/content/(?P<extracted_part>.*?)\.pdf$', file)
  if fn:
    fname.append(fn.group('extracted_part'))
  texts.append(text_extraction(file))

In [60]:
import nltk

# Download the punkt tokenizer if not already installed
nltk.download('punkt')

def split_into_sentences(items):
  sentences = []
  for item in items:
    sentences += (nltk.sent_tokenize(item))
  return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [61]:
sentences =[]
for items in texts:
    sentences.append(split_into_sentences(items))

In [12]:
!pip install -q transformers
from transformers import pipeline

In [13]:
model_name ='mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
sentiment_pipeline = pipeline("sentiment-analysis", model = model_name)

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [66]:
def namingFunc(pdf):
  # Find the index of the last '/' and the last occurrence of '.pdf'
  # last_slash_index = pdf.rfind('/')
  # last_dot_pdf_index = pdf.rfind('.pdf')
  parts = pdf.split('/')
  # Extract the desired text between the last '/' and the last 4 characters
  compID = parts[-4] # pdf[last_slash_index + 1:last_dot_pdf_index]
  qurtr = parts[-2]
  year = parts[-3]
  print(pdf)
  return [compID, year, qurtr]

In [67]:
c=0
for items,name in zip(sentences,fname):
  c+=1
  names = namingFunc(name)
  sent_count = len(items)
  sentiment_list = []
  positive_sent = []
  negative_sent = []
  neutral_sent = []

  for item in items:
    sentiment = sentiment_pipeline(item)[0]['label']
    sentiment_list.append (sentiment)

    if sentiment == 'positive':
      positive_sent.append(item)
    elif sentiment == 'negative':
      negative_sent.append(item)
    else:
      neutral_sent.append(item)

  neutral = sentiment_list.count('neutral')/sent_count
  positive = sentiment_list.count ('positive')/sent_count
  negative = sentiment_list.count ('negative')/sent_count
  score = [positive, neutral, negative]
  if os.path.exists(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/"):
    pass
  else:
    os.makedirs(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/")
  with open(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/{names[0]}_SCO.txt","w+") as fp:
    for i in score:
        fp.write(str(i)+"\n")
  with open(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/{names[0]}_POS.txt","w+") as fp:
    for i in positive_sent:
      i = re.sub(r"\n+"," ",i)
      fp.write(i+"\n")
  with open(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/{names[0]}_NEG.txt","w+") as fp:
    for i in negative_sent:
      i = re.sub(r"\n+"," ",i)
      fp.write(i+"\n")
  with open(f"/content/drive/MyDrive/fin/{names[0]}/{names[1]}/{names[2]}/sentiment/{names[0]}_NEU.txt","w+") as fp:
    for i in neutral_sent:
      i = re.sub(r"\n+"," ",i)
      fp.write(i+"\n")
  print(c)

drive/MyDrive/fin/TATASTEEL/2023/4/TATASTEEL
1
drive/MyDrive/fin/TATASTEEL/2023/3/TATASTEEL
2
drive/MyDrive/fin/TATASTEEL/2024/1/TATASTEEL
3
drive/MyDrive/fin/TATASTEEL/2024/2/TATASTEEL
4
drive/MyDrive/fin/TATAMOTORS/2024/2/TATAMOTORS
5
drive/MyDrive/fin/TATAMOTORS/2024/1/TATAMOTORS
6
drive/MyDrive/fin/TATAMOTORS/2023/4/TATAMOTORS
7
drive/MyDrive/fin/TATAMOTORS/2023/3/TATAMOTORS
8
drive/MyDrive/fin/ASHOKLEY/2024/1/ASHOKLEY
9
drive/MyDrive/fin/ASHOKLEY/2023/4/ASHOKLEY
10
drive/MyDrive/fin/ASHOKLEY/2023/3/ASHOKLEY
11
drive/MyDrive/fin/WIPRO/2023/3/WIPRO
12
drive/MyDrive/fin/WIPRO/2023/4/WIPRO
13
drive/MyDrive/fin/WIPRO/2024/1/WIPRO
14
drive/MyDrive/fin/WIPRO/2024/2/WIPRO
15
drive/MyDrive/fin/TCS/2023/1/TCS
16
drive/MyDrive/fin/TCS/2023/2/TCS
17
drive/MyDrive/fin/TCS/2023/4/TCS
18
drive/MyDrive/fin/TCS/2023/3/TCS
19
drive/MyDrive/fin/DLF/2023/4/DLF
20
drive/MyDrive/fin/LTTS/2024/2/LTTS
21
drive/MyDrive/fin/LTTS/2024/1/LTTS
22
drive/MyDrive/fin/LTTS/2023/4/LTTS
23
drive/MyDrive/fin/LTTS/20

In [ ]:
drive

In [18]:
import shutil

shutil.rmtree('/content/drive')